# Paper Savior with LionAGI and LlamaIndex Vector Index

-- how to do auto explorative research with LionAGI plus RAG using llamaindex Vector Index & embedding 

- [LionAGI](https://github.com/lion-agi/lionagi)
- [LlamaIndex](https://www.llamaindex.ai)

In [1]:
# %pip install lionagi llama_index

In [2]:
query = 'Large Language Model Time Series Analysis'
dir = "data/log/researcher/"
num_papers = 5

### 1. Build a Vector Index with llama_index

In [3]:
from llama_index import download_loader, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.node_parser import SentenceSplitter

In [4]:
ArxivReader = download_loader("ArxivReader")
loader = ArxivReader()
node_parser = SentenceSplitter(chunk_size=800, chunk_overlap=50)

# let us download some papers from arvix
documents, abstracts = loader.load_papers_and_abstracts(search_query=query, max_results=num_papers)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=False)

# set up index object
llm = OpenAI(temperature=0.1, model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)
index1 = VectorStoreIndex(nodes, include_embeddings=True, service_context=service_context)

# set up query engine
query_engine = index1.as_query_engine(include_text=False, response_mode="tree_summarize")

In [5]:
len(abstracts)

5

### 2. Write a tool description according to OpenAI schema

In [6]:
import lionagi as li

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_arvix_papers",
            "description": "Perform a query to a QA bot with access to a knowledge graph index built with papers from arvix",
            "parameters": {
                "type": "object",
                "properties": {
                    "str_or_query_bundle": {
                        "type": "string",
                        "description": "a question to ask the QA bot",
                    }
                },
                "required": ["str_or_query_bundle"],
            },
        }
    }
]

func = query_engine.query

In [8]:
system = {
    "persona": "a helpful world-class researcher",
    "requirements": "think step by step before returning a clear, precise worded answer with a humble yet confident tone",
    "responsibilities": f"you are asked to help with researching on the topic of {query}",
    "tools": "provided with a QA bot for grounding responses"
}

### 3. Research: PROMPTS

#### FORMATS

In [9]:
deliver_format1 = {"return required": "yes", "return format": "paragraph"}

deliver_format2 = {"return required": "yes", 
    "return format": {
        "type": "json_mode", 
        "format": {
            'paper': "paper_name",
            "summary": "...", 
            "research question": "...", 
            "talking points": {
                "point 1": "...",
                "point 2": "...",
                "point 3": "..."
            }}}},



#### PROMPTS

In [10]:
function_call = {
    "notice":"At each task step, identified by step number, you must use the tool at least twice. Notice you are provided with a QA bot as your tool, the bot has access to the papers via a queriable index that takes natural language query and return a natural language answer. You can decide whether to invoke the function call, you will need to ask the bot when there are things need clarification or further information. you provide the query by asking a question, please use the tool extensively as you can",}

instruct1 = {
    "task step": "1", 
    "task name": "read paper abstracts", 
    "task objective": "get initial understanding of the papers of interest", 
    "task description": "provided with abstracts of paper, provide a brief summary highlighting the paper core points, the purpose is to extract as much information as possible",
    "deliverable": deliver_format1
}

instruct2 = {
    "task step": "2",
    "task name": "propose research questions and talking points", 
    "task objective": "initial brainstorming", 
    "task description": "from the improved understanding of the paper, please propose an interesting, unique and practical research questions, support your reasoning. Kept on asking questions if things are not clear",
    "deliverable": deliver_format2,
    "function calling": function_call
}

### 4. Research: Setup Workflow

In [11]:
abstracts = li.l_call(range(len(abstracts)), lambda i: abstracts[i].text)

In [12]:
async def read_propose(context, num=5):
    researcher = li.Session(system, dir=dir)
    researcher.register_tools(tools, func)
    
    await researcher.initiate(instruct1, context=context, temperature=0.7)
    await researcher.auto_followup(instruct2, tools=tools, num=num, tool_parser=lambda x: x.response)
    
    researcher.messages_to_csv()
    researcher.log_to_csv()
    return researcher

### 5. Research: Run the workflow

In [13]:
researchers = await li.al_call(abstracts, read_propose)

11 logs saved to data/log/researcher/_messages_2023-12-14T22_31_25_200017.csv
4 logs saved to data/log/researcher/_llmlog_2023-12-14T22_31_25_201684.csv
11 logs saved to data/log/researcher/_messages_2023-12-14T22_31_41_607578.csv
4 logs saved to data/log/researcher/_llmlog_2023-12-14T22_31_41_608202.csv
11 logs saved to data/log/researcher/_messages_2023-12-14T22_32_35_107945.csv
4 logs saved to data/log/researcher/_llmlog_2023-12-14T22_32_35_109455.csv
11 logs saved to data/log/researcher/_messages_2023-12-14T22_33_02_227138.csv
4 logs saved to data/log/researcher/_llmlog_2023-12-14T22_33_02_230544.csv
14 logs saved to data/log/researcher/_messages_2023-12-14T22_33_36_715894.csv
5 logs saved to data/log/researcher/_llmlog_2023-12-14T22_33_36_716527.csv


In [14]:
out1 = li.l_call(researchers, lambda x: x.conversation.messages[-1]['content'])

In [15]:
from IPython.display import Markdown

Markdown(out1[0])

Based on the information retrieved from the QA bot and the understanding of the paper's abstract, we can now propose a research question and formulate some talking points.

Research Question:
How can the adaptive segmentation strategy of the LPTM model be further improved to handle time series data with highly irregular patterns or missing data, which are common in real-world scenarios?

Talking Points:
1. Point 1: Compare the adaptive segmentation strategy of LPTM with traditional fixed and heuristic-based methods, focusing on their adaptability and performance in real-world time series data with irregularities and gaps.
2. Point 2: Investigate the impact of different types of noise and outliers on the segmentation and subsequent performance of the LPTM model. This includes exploring robustness measures that can be integrated into the model.
3. Point 3: Consider the potential of LPTM in domains with sparse or limited data availability, and discuss methodologies to leverage transfer learning or few-shot learning techniques to enhance its performance in such settings.

To support the proposed research question and the talking points, it is understood that time series data in real-world applications often present challenges not accounted for in controlled environments, such as irregular sampling rates, missing data, and noise. Thus, enhancing the adaptability and robustness of the LPTM model's segmentation strategy could lead to more accurate and reliable performance across a broader range of domains and datasets.

If further clarification or information is needed to refine these points, I can continue to use the QA bot tool to ask relevant questions.

In [16]:
Markdown(out1[1])

Based on the information provided and the additional insights gained from querying the QA bot, I propose the following research question and talking points for the paper "Time-LLM: Time Series Forecasting by Reprogramming Large Language Models":

Research Question:
How can Time-LLM be adapted to handle high-noise environments, such as social media trend analysis or economic indicators during periods of high uncertainty, and what mechanisms can be implemented to ensure robustness against overfitting in such volatile domains?

Talking Points:
- Point 1: The potential of Time-LLM to be fine-tuned for domains with high-frequency, high-volume, and noisy data, like stock market predictions, considering its success in few-shot and zero-shot learning scenarios.
- Point 2: The need for strategies within Time-LLM to manage and filter out noise in data, ensuring that the forecasts are based on signal rather than noise, which is particularly challenging in domains like social media trend analysis.
- Point 3: The importance of investigating the model's ability to generalize across different time series datasets and its robustness in the face of overfitting, especially when optimized on one dataset and evaluated on another.

These talking points can guide further exploration into the capabilities and limitations of Time-LLM, with the goal of extending the model's applicability to a wider range of real-world forecasting challenges.

{
    "paper": "Time-LLM: Time Series Forecasting by Reprogramming Large Language Models",
    "summary": "The paper introduces Time-LLM, a framework for time series forecasting by reprogramming large language models without altering the original models. Time-LLM uses a novel Prompt-as-Prefix technique to enhance reasoning with time series data and has demonstrated superiority over specialized models in various learning scenarios.",
    "research question": "How can Time-LLM be adapted to handle high-noise environments, such as social media trend analysis or economic indicators during periods of high uncertainty, and what mechanisms can be implemented to ensure robustness against overfitting in such volatile domains?",
    "talking points": {
        "point 1": "Exploring the adaptability of Time-LLM to high-frequency, high-volume, and noisy data domains like stock markets.",
        "point 2": "Investigating noise management strategies for Time-LLM in complex, noisy data environments.",
        "point 3": "Assessing the generalization and robustness of Time-LLM to prevent overfitting when applied to diverse time series forecasting tasks."
    }
}

In [17]:
Markdown(out1[2])

Based on the understanding gained from the paper summary and the additional insights provided by the QA bot, we can formulate the following research question and talking points:

**Research Question:**

How can the domain-specific limitations of pre-trained NLP and CV models be addressed to enhance their performance and applicability in diverse time series analysis tasks?

**Talking Points:**

1. **Data Dynamics Alignment**: Investigate methods to align the pre-trained models' understanding of data dynamics with the unique temporal characteristics of time series data, such as trends and seasonality.

2. **Temporal Context Integration**: Explore architectural modifications or training techniques that can integrate temporal context into pre-trained models, enabling them to handle sequential data with temporal dependencies more effectively.

3. **Domain-Specific Adaptation**: Develop strategies for adapting pre-trained models to capture domain-specific features of time series data, which may involve reprogramming input and output layers or introducing new embeddings that are indicative of time-related aspects.

The proposed research question aims to tackle the challenge of transferring knowledge from pre-trained models in NLP and CV to time series analysis, which is not straightforward due to differences in data types and task objectives. The suggested talking points pave the way for a more detailed examination of potential solutions and methodologies that could bridge the gap between these domains and make pre-trained models more effective for time series analysis tasks.

```json
{
  "paper": "One Fits All: Power General Time Series Analysis by Pretrained LM",
  "summary": "The paper examines the application of pre-trained NLP and CV models to time series analysis tasks. It introduces the Frozen Pretrained Transformer (FPT) and demonstrates its effectiveness across various tasks. It also suggests that the self-attention mechanism acts similarly to PCA, offering insights into the transformer's adaptability to time series data.",
  "research question": "How can the domain-specific limitations of pre-trained NLP and CV models be addressed to enhance their performance and applicability in diverse time series analysis tasks?",
  "talking points": {
    "point 1": "Data Dynamics Alignment: Addressing the challenge of aligning data dynamics of pre-trained models with time series characteristics.",
    "point 2": "Temporal Context Integration: Modifying architectures to incorporate temporal context and dependencies in sequential data.",
    "point 3": "Domain-Specific Adaptation: Adapting models to capture domain-specific features and temporal aspects of time series data."
  }
}
```

In [18]:
Markdown(out1[3])

Based on the improved understanding of the paper and the limitations highlighted by the function call results, I propose the following research question and talking points:

**Research Question:**
- How can the decoder-only foundation model for time-series forecasting be adapted to better handle anomalies and irregularities in the data, which are common in real-world scenarios?

**Talking Points:**
- **Point 1:** Exploration of techniques for anomaly detection and correction within the model's architecture to improve its robustness and accuracy in real-world applications.
- **Point 2:** Investigation into the impact of data diversity and quality in the pretraining phase on the model's ability to generalize across different datasets with various characteristics.
- **Point 3:** Development of methods to enhance the model's capacity to deal with varying history lengths, prediction lengths, and temporal granularities without compromising performance.

The research question is practical and unique as it addresses a real-world challenge of dealing with anomalies in time-series data, which can significantly affect forecasting accuracy. The talking points support this question by focusing on potential areas of enhancement for the model.

To ensure clarity and to gather additional insights that could further refine the research question and talking points, I will invoke another function call.

{"function": "func_query_arvix_papers", "arguments": "{\"str_or_query_bundle\":\"Can the decoder-only foundation model be fine-tuned to specific domains or datasets to improve its performance on tasks with high levels of noise or non-standard patterns?\"}"}

In [19]:
Markdown(out1[4])

Based on the information provided and the capabilities of the Multimodal Transformer (MulT), we can propose the following research question and talking points:

```json
{
  "paper": "Multimodal Transformer for Unaligned Multimodal Language Sequences",
  "summary": "The Multimodal Transformer (MulT) is a model designed to process multimodal human language data by addressing non-alignment due to variable sampling rates and capturing long-range dependencies across different modalities using a directional pairwise crossmodal attention mechanism. This model surpasses existing methods in handling multimodal time-series data and shows promise in capturing correlated signals across modalities.",
  "research question": "How can the directional pairwise crossmodal attention mechanism in MulT be adapted for real-time processing in applications such as simultaneous translation or live emotional feedback systems?",
  "talking points": {
    "point 1": "Investigate the real-time adaptability of MulT for applications requiring immediate feedback, such as live emotional analysis during interactive sessions or negotiations.",
    "point 2": "Explore how the MulT model can be fine-tuned for specific applications like social robotics, where understanding and responding to human cues quickly is critical.",
    "point 3": "Assess the potential of integrating MulT with edge computing devices to enhance real-time processing capabilities, particularly for Internet of Things (IoT) applications involving human interaction."
  }
}
```

The proposed research question is aimed at exploring the practicality of MulT in scenarios that demand immediate processing and feedback. The talking points support the reasoning by discussing the significance of real-time processing in various interactive applications, the customization of the model for specialized use cases, and the integration with modern computing paradigms such as edge computing, which are relevant for IoT scenarios.